In [ ]:
import os
import requests
from typing import Dict, Any

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage

# Globals

In [49]:
MODEL = "avathon-openai/gpt-4.1" #gpt-4o-mini
RECURSION_LIMIT = 100
BASE_URL = "https://model-gateway.dev-platform.avathon.com"
# Load api keys from env file
from dotenv import load_dotenv
load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
MODEL_API_KEY = os.getenv("MODEL_API_KEY")

# Define model

In [51]:
llm = ChatOpenAI(model=MODEL, base_url=BASE_URL, api_key=MODEL_API_KEY)

Test model

In [52]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to German. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg.content

'Ich liebe das Programmieren.'

# TOOL CALL

## Weather Agent

LLM Agent that provides the current weather at specified location. 

This agent uses the api call with structured response.

In [53]:
from pydantic import BaseModel, Field

class GetWeatherResponse(BaseModel):
    location: str = Field(description="The location of the weather")
    temperature: str = Field(description="The temperature of the weather")
    feels_like: str = Field(description="The feels like temperature of the weather")
    description: str = Field(description="The description of the weather")
    humidity: str = Field(description="The humidity of the weather")
    wind_speed: str = Field(description="The wind speed of the weather")


@tool
def get_weather(location: str) -> GetWeatherResponse:
    """Get current weather information for a specific location.
    
    Args:
        location: City name or location to get weather for
        
    Returns:
        Weather information as a formatted string
    """
    try:
        url = "http://api.weatherapi.com/v1/current.json"
        params = {
            "q": location,
            "key": WEATHER_API_KEY,
            "aqi": "no"
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        weather_info = GetWeatherResponse(
            location=f"{data['location']['name']}, {data['location']['country']}",
            temperature=f"{data['current']['temp_c']}°C",
            feels_like=f"{data['current']['feelslike_c']}°C",
            description=data['current']['condition']['text'],
            humidity=f"{data['current']['humidity']}%",
            wind_speed=f"{data['current']['wind_kph']} km/h"
        )
        
        return weather_info
        
    except requests.exceptions.RequestException as e:
        return f"Error fetching weather data: {str(e)}"
    except KeyError as e:
        return f"Error parsing weather data: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"


# Alternate weather api (works without api key)
def _get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']



print(get_weather("Bengaluru"))

location='Bengaluru, India' temperature='20.4°C' feels_like='20.4°C' description='Partly cloudy' humidity='94%' wind_speed='23.4 km/h'


In [57]:
def weather_agent(user_query: str) -> str:
    """Simple weather agent that can answer weather-related questions.
    
    Args:
        user_query: User's question about weather
        
    Returns:
        Agent's response including weather data if requested
    """
    messages = [
        SystemMessage(content="You are a helpful weather assistant. When users ask about weather, use the get_weather tool to provide current weather information. Be conversational and helpful."),
        HumanMessage(content=user_query)
    ]
    
    ai_msg = llm_with_tools.invoke(messages)
    # print("DEBUG weather agent: ", ai_msg)
    
    if ai_msg.tool_calls:
        messages.append(ai_msg)
        
        for tool_call in ai_msg.tool_calls:
            selected_tool = {"get_weather": get_weather}[tool_call["name"].lower()]
            tool_output = selected_tool.invoke(tool_call["args"])
            messages.append(ToolMessage(content=tool_output, tool_call_id=tool_call["id"]))
        
        ai_msg = llm_with_tools.invoke(messages)
    
    return ai_msg.content


In [54]:
tools = [get_weather]
llm_with_tools = llm.bind_tools(tools)

print("Weather agent is ready! The LLM can now access weather data.")


Weather agent is ready! The LLM can now access weather data.


Spot check Weather agent

In [55]:
user_question = "What's the weather like in San Francisco?"

ai_msg = llm_with_tools.invoke(user_question)
# print(ai_msg)

messages = []
if ai_msg.tool_calls:
    messages.append(ai_msg)
    for tool_call in ai_msg.tool_calls:
        print(tool_call)
        selected_tool = {"get_weather": get_weather}[tool_call["name"].lower()]
        # print(selected_tool)
        tool_output = selected_tool.invoke(tool_call["args"])
        # print(tool_output)
        messages.append(ToolMessage(content=tool_output, tool_call_id=tool_call["id"]))

    ai_msg = llm_with_tools.invoke(messages)
    print(ai_msg.content)




{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_b6nxtIpnAIndFCuT5zewIiym', 'type': 'tool_call'}
The current weather in San Francisco is partly cloudy, with a temperature of 16.3°C (feels like 16.3°C). Humidity is at 94%, and the wind speed is 23.0 km/h.


## Testing the Weather Agent

**Note**: You need to:
1. Get a free API key from [OpenWeatherMap](https://openweathermap.org/api)
2. Replace `WEATHER_API_KEY` with your actual API key
3. Install required package: `uv pip install requests`


In [58]:
test_queries = [
    "What's the weather like in London?",
    "How is the weather in Tokyo today?",
    "Tell me about the current weather conditions in New York",
    "Is it raining in Paris right now?",
    "What should I wear outside in Mumbai today?"
]

print("Testing Weather Agent:")
print("=" * 50)

for query in test_queries:
    print(f"\n🤔 User: {query}")
    try:
        response = weather_agent(query)
        print(f"🤖 Agent: {response}")
    except Exception as e:
        print(f"❌ Error: {e}")
    print("-" * 30)


Testing Weather Agent:

🤔 User: What's the weather like in London?
🤖 Agent: The weather in London is currently overcast with a temperature of 17.3°C. It feels about the same as the actual temperature. Humidity is at 82%, and there's a gentle breeze with a wind speed of 7.9 km/h.

Would you like a forecast for the next few days or any other weather details?
------------------------------

🤔 User: How is the weather in Tokyo today?
🤖 Agent: The weather in Tokyo today is sunny, with a temperature of 30.2°C and it feels like 32.0°C. Humidity is at 51%, and there's a gentle breeze with wind speeds of 15.8 km/h.

Let me know if you need more details or want the forecast for the coming days!
------------------------------

🤔 User: Tell me about the current weather conditions in New York
🤖 Agent: Currently in New York, it's partly cloudy with a temperature of 32.8°C, but it feels warmer at 36.1°C. Humidity is at 47%, and there's a moderate wind blowing at 19.1 km/h. Would you like more details